In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import logging
from dotenv import load_dotenv

In [2]:
load_dotenv()

BOT_TOKEN = os.getenv('BOT_TOKEN')
BASE_URL = "https://finder.work/vacancies/project?categories=1"
JSON_FILE = "vacancies.json"

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [3]:
def load_vacancies():
    """Загружает сохранённые вакансии из JSON-файла."""
    try:
        with open(JSON_FILE, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        return {}

def save_vacancies(vacancies):
    """Сохраняет вакансии в JSON-файл."""
    with open(JSON_FILE, 'w', encoding='utf-8') as f:
        json.dump(vacancies, f, ensure_ascii=False, indent=4)

In [4]:
def parse_vacancies():
    """Парсит вакансии со всех страниц."""
    vacancies = {}
    page = 1
    while True:
        url = f"{BASE_URL}&page={page}" if page > 1 else BASE_URL
        try:
            response = requests.get(url, timeout=10)
            if response.status_code != 200:
                logger.warning(f"Не удалось загрузить страницу {url}: статус {response.status_code}")
                break

            soup = BeautifulSoup(response.text, 'html.parser')
            vacancy_cards = soup.find_all('div', class_='fui-flex fui-flex-col fui-bg-white fui-sheet-shadow_sm fui-card transition-shadow cursor-pointer')

            if not vacancy_cards:
                logger.info(f"Вакансии не найдены на странице {page}")
                break

            for card in vacancy_cards:
                title_elem = card.find('a', class_='fui-no-underline font-bold text-lg w-0 grow flex-wrap text-blue-dark')
                title = title_elem.text.strip() if title_elem else "N/A"
                vacancy_url = title_elem['href'] if title_elem else ""
                if vacancy_url and not vacancy_url.startswith('http'):
                    vacancy_url = "https://finder.work" + vacancy_url
                
                salary_elem = card.find('div', class_='font-bold text-black text-xl')
                salary = salary_elem.text.replace('\xa0', ' ') if salary_elem else "N/A"
                
                company_elem = card.find('a', class_='fui-no-underline text-grey-dark')
                company = company_elem.text if company_elem else "N/A"

                if vacancy_url:  # Используем URL как ключ, если он есть
                    vacancies[vacancy_url] = {
                        'title': title,
                        'salary': salary,
                        'company': company,
                        'url': vacancy_url
                    }

            page += 1
        except requests.RequestException as e:
            logger.error(f"Ошибка при запросе страницы {url}: {e}")
            break

    return vacancies

In [5]:
vacancies = parse_vacancies()

2025-07-23 20:55:01,025 - ERROR - Ошибка при запросе страницы https://finder.work/vacancies/project?categories=1: [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:2578)


In [6]:
vacancies

{}

In [7]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обрабатывает команду /start: парсит вакансии, сохраняет и отправляет файл."""
    chat_id = update.message.chat_id
    logger.info(f"Получена команда /start от чата {chat_id}")

    await context.bot.send_message(chat_id=chat_id, text="Запускаю парсинг вакансий...")

    # Парсим вакансии
    vacancies = parse_vacancies()

    if not vacancies:
        await context.bot.send_message(chat_id=chat_id, text=\
                                       "Не удалось найти вакансии. Возможно, проблема с сайтом.")
        return

    # Сохраняем вакансии в файл
    save_vacancies(vacancies)
    await context.bot.send_message(chat_id=chat_id, text=\
                                   f"Найдено {len(vacancies)} вакансий. Сохраняю в {JSON_FILE} и отправляю файл.")

    # Отправляем файл пользователю
    try:
        with open(JSON_FILE, 'rb') as f:
            await context.bot.send_document(chat_id=chat_id, document=f, filename=JSON_FILE)
    except Exception as e:
        logger.error(f"Ошибка при отправке файла: {e}")
        await context.bot.send_message(chat_id=chat_id, text="Ошибка при отправке файла.")

In [8]:
async def check_vacancies(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обрабатывает команду /check: сравнивает вакансии и отправляет новые."""
    chat_id = update.message.chat_id
    logger.info(f"Получена команда /check от чата {chat_id}")

    await context.bot.send_message(chat_id=chat_id, text="Проверяю вакансии...")

    # Парсим текущие вакансии
    old_vacancies = load_vacancies()
    new_vacancies = parse_vacancies()

    if not new_vacancies:
        await context.bot.send_message(chat_id=chat_id, text=\
                                       "Не удалось найти вакансии. Возможно, проблема с сайтом.")
        return

    # Проверяем новые вакансии
    new_vacancy_found = False
    for url, vacancy in new_vacancies.items():
        if url not in old_vacancies:
            message = f"Новая вакансия!\n\nНазвание: {vacancy['title']}\n\
                Зарплата: {vacancy['salary']}\nОрганизация: {vacancy['company']}\nСсылка: {url}"
            await context.bot.send_message(chat_id=chat_id, text=message)
            new_vacancy_found = True

    if not new_vacancy_found:
        await context.bot.send_message(chat_id=chat_id, text="Новых вакансий не найдено.")

    # Сохраняем новые данные
    save_vacancies(new_vacancies)

In [9]:
async def handle_file(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обрабатывает загруженный JSON-файл с вакансиями."""
    chat_id = update.message.chat_id
    file = update.message.document

    if not file.mime_type == 'application/json':
        await context.bot.send_message(chat_id=chat_id, text=\
                                       "Пожалуйста, загрузите файл в формате JSON.")
        return

    try:
        # Получаем файл
        file_obj = await file.get_file()
        file_path = await file_obj.download_to_drive(JSON_FILE)

        # Проверяем корректность JSON
        with open(JSON_FILE, 'r', encoding='utf-8') as f:
            json.load(f)  # Проверяем, что файл валидный JSON

        await context.bot.send_message(chat_id=chat_id, text=\
                                       f"Файл {JSON_FILE} успешно загружен и будет использован для сравнения при следующей команде /check.")
    except Exception as e:
        logger.error(f"Ошибка при обработке файла: {e}")
        await context.bot.send_message(chat_id=chat_id, text=\
                                       "Ошибка при загрузке файла. Убедитесь, что это валидный JSON-файл.")

In [12]:
async def get_file(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.message.chat_id
    if os.path.exists(JSON_FILE):
        with open(JSON_FILE, 'rb') as f:
            await context.bot.send_document(chat_id=chat_id, document=f, filename=JSON_FILE)
    else:
        await context.bot.send_message(chat_id=chat_id, text="Файл vacancies.json не найден.")

In [11]:
application = Application.builder().token(BOT_TOKEN).build()

In [14]:
application.add_handler(CommandHandler("start", start))
application.add_handler(CommandHandler("check", check_vacancies))
application.add_handler(CommandHandler("getfile", get_file))
application.add_handler(MessageHandler(filters.Document.MimeType("application/json"), handle_file))

In [ ]:
def main():
    """Запускает бота и проверяет наличие файла при старте."""
    application = Application.builder().token(BOT_TOKEN).build()

    # Добавляем обработчики
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("check", check_vacancies))
    application.add_handler(CommandHandler("getfile", get_file))
    application.add_handler(MessageHandler(filters.Document.MimeType("application/json"), handle_file))

    # Проверяем, существует ли файл vacancies.json при запуске
    if not os.path.exists(JSON_FILE):
        logger.info("Файл vacancies.json не найден, парсим вакансии при запуске")
        vacancies = parse_vacancies()
        if vacancies:
            save_vacancies(vacancies)
            logger.info(f"Сохранено {len(vacancies)} вакансий в {JSON_FILE}")
        else:
            logger.warning("Не удалось найти вакансии при первом запуске")

    # Запускаем бота
    logger.info("Бот запущен")
    application.run_polling()